In [6]:
import pandas as pd

In [7]:
import sys
sys.path.append('../../src')
from DataHandler import DataHandler

In [10]:
dataHandler = DataHandler("../config.yaml")

In [18]:
df_aimeta = dataHandler.load_data("data_files.raw_data.df_aimeta")
df_aimeta.head(5)

📂 Lade: OPAL_ai_meta.p
   ✅ Geladen: 4,548 Zeilen × 25 Spalten


,pipe:ID,pipe:file_type,ai:author,ai:revisedAuthor,ai:affilation,ai:title,ai:type,ai:keywords_ext,ai:keywords_gen,ai:keywords_dnb,...,ai:keywords_dnb_ids,ai:keywords_ext_ids_valid,ai:keywords_ext_valid_ratio,ai:keywords_dnb_ids_valid,ai:keywords_dnb_valid_ratio,ai:keywords_gen_ids_valid,ai:keywords_gen_valid_ratio,all_ddc_valid_ids,ddc_valid_count,valid_ddc_keywords
0,8I6sM5zapD60,pdf,"Stephan Gerhold, Marcel Beyer",[Vorname='Stephan' Familienname='Gerhold' Tite...,,Übung 3 Photogrammetrie,Aufgabenblatt,"Photogrammetrie, digitale Entzerrung, Resampli...","Photogrammetrie, digitale Entzerrung, Resampli...","Photogrammetrie, Bildverarbeitung, Digitale En...",...,"[0, 1, 2, 9, 11, 12, 14, 15, 16, 17, 18]","[0, 1, 4, 14]",0.333333,"[0, 1, 11, 12, 14, 16]",0.545455,"[0, 1, 4, 11, 12, 14]",0.545455,"[0, 1, 4, 14, 0, 1, 11, 12, 14, 16, 0, 1, 4, 1...",16,"[Photogrammetrie, digitale Entzerrung, Paralle..."
1,8ZICOHBmAHyQ,pdf,Keine Angabe.,[],,3.1 Grundlagen der Tragwerke,Skript,"Tragwerke, Lagerreaktionen, Gleichgewichtsbedi...","Tragwerke, Lagerreaktionen, Gleichgewichtsbedi...","Statik, Tragwerke, Mechanik, Festigkeitslehre,...",...,"[34, 35, 36, 37, 38, 19, 20, 21, 22, 9079, 24]","[19, 24, 27, 28]",0.363636,"[34, 35, 36, 37, 38, 19, 24]",0.636364,"[33, 19, 27, 30]",0.363636,"[19, 24, 27, 28, 34, 35, 36, 37, 38, 19, 24, 3...",15,"[Tragwerke, Festigkeitslehre, Freiheitsgrade, ..."
2,8Lfz8SAKa6k0,pdf,Robert Ringel,[Vorname='Robert' Familienname='Ringel' Titel=''],,Gedanken und Notizen zu unserem Mini-KI-Worksh...,Protokoll,"Künstliche Intelligenz, Automatisierung, Indus...","Künstliche Intelligenz, Automatisierung, Indus...","Künstliche Intelligenz, Automatisierung, Indus...",...,"[39, 40, 49, 52, 55, 54, 3191, 56, 3193, 58, 5...","[39, 40, 41, 42, 43, 45, 46, 47, 3191]",0.750000,"[39, 40, 49, 52, 55, 54, 3191, 56, 3193, 58, 5...",1.000000,"[39, 40, 43, 45, 47, 49, 52, 3191]",0.666667,"[39, 40, 41, 42, 43, 45, 46, 47, 3191, 39, 40,...",29,"[Künstliche Intelligenz, Automatisierung, Indu..."
3,3ztCv-WpxJ4U,pdf,Norbert Engemaier,[Vorname='Norbert' Familienname='Engemaier' Ti...,Technische Universität Dresden,Allgemeine Anforderungen,Aufgabenblatt,"Bibliographie, Literaturangaben, Sekundärquell...","wissenschaftliches Arbeiten, akademische Schre...","Wissenschaftliches Arbeiten, akademische Schri...",...,"[33893, 70, 6281, 74, 75, 76, 77, 78, 79, 80, ...","[65, 66, 67, 68, 60]",0.500000,"[70, 74, 75, 76, 60]",0.384615,"[66, 67, 70, 72, 74, 76, 75, 60]",0.666667,"[65, 66, 67, 68, 60, 70, 74, 75, 76, 60, 66, 6...",18,"[Referat, Hausarbeit, Essay, Mündliche Prüfung..."
4,6mOhjfscZK2A,pdf,Keine Angabe.,[],Technische Universität Dresden,"Vorlesung Technische Mechanik I - AGBF, TU Dre...",Vorlesungsfolien,"Technische Mechanik, Starrer Körper, Einzelmom...","Technische Mechanik, Starrer Körper, Kraft, Mo...","Technische Mechanik, Starrer Körper, Kraft, Mo...",...,"[34, 82, 83, 86, 88, 89, 92, 93, 94, 95]","[82, 83, 88, 89, 90, 30]",0.600000,"[34, 82, 83, 88, 89, 93, 94]",0.700000,"[82, 83, 88, 89, 90]",0.500000,"[82, 83, 88, 89, 90, 30, 34, 82, 83, 88, 89, 9...",18,"[Technische Mechanik, Starrer Körper, Kraft, M..."


In [30]:
# UMFASSENDE DATENBEREINIGUNG DER DEWEY-KLASSIFIKATIONEN
print("=== UMFASSENDE DATENBEREINIGUNG ===\n")

import re

# Definition problematischer Labels und Patterns
problematic_patterns = [
    "(Keine passende Kategorie)",
    "(Keine passende Kategorie, nächstliegende: ",
    "Keine passende Kategorie",
    # Weitere problematische Patterns können hier hinzugefügt werden
]

def clean_dewey_comprehensive(dewey_list):
    """
    Umfassende Bereinigung der Dewey-Einträge:
    1. Entfernt problematische Labels
    2. Bereinigt Anführungszeichen und andere Formatierungsprobleme
    3. Normalisiert die Label-Texte
    """
    if not isinstance(dewey_list, list):
        return dewey_list
    
    cleaned_list = []
    
    for entry in dewey_list:
        if isinstance(entry, dict) and 'label' in entry and 'notation' in entry:
            original_label = entry['label']
            
            # 1. Prüfe auf problematische Labels
            is_problematic = False
            for pattern in problematic_patterns:
                if pattern in original_label:
                    is_problematic = True
                    break
            
            if is_problematic:
                continue  # Überspringe problematische Einträge
            
            # 2. Bereinige das Label von Anführungszeichen und anderen Problemen
            cleaned_label = original_label
            
            # Entferne verschiedene Arten von Anführungszeichen
            cleaned_label = re.sub(r'^["\'""''„""''`´]+|["\'""''„""''`´]+$', '', cleaned_label)
            
            # Entferne doppelte Anführungszeichen im Text
            cleaned_label = re.sub(r'["\'""''„""''`´]{2,}', '', cleaned_label)
            
            # Normalisiere Leerzeichen
            cleaned_label = re.sub(r'\s+', ' ', cleaned_label).strip()
            
            # Entferne führende/nachgestellte Punkte und Kommas
            cleaned_label = re.sub(r'^[.,\s]+|[.,\s]+$', '', cleaned_label)
            
            # Erstelle bereinigten Eintrag
            cleaned_entry = entry.copy()
            cleaned_entry['label'] = cleaned_label
            cleaned_entry['original_label'] = original_label  # Behalte Original für Referenz
            
            # Nur hinzufügen wenn das bereinigte Label nicht leer ist
            if cleaned_label.strip():
                cleaned_list.append(cleaned_entry)
    
    return cleaned_list

# Statistiken vor der Bereinigung
print("STATISTIKEN VOR DER BEREINIGUNG:")
original_count = sum(len(dewey_list) for dewey_list in df_aimeta['ai:dewey'] if isinstance(dewey_list, list))
print(f"  Ursprüngliche Dewey-Einträge: {original_count}")

# Analysiere Anführungszeichen-Probleme
labels_with_quotes = 0
for dewey_list in df_aimeta['ai:dewey']:
    if isinstance(dewey_list, list):
        for entry in dewey_list:
            if isinstance(entry, dict) and 'label' in entry:
                label = entry['label']
                if re.search(r'["\'""''„""''`´]', label):
                    labels_with_quotes += 1

print(f"  Labels mit Anführungszeichen: {labels_with_quotes}")

# Wende umfassende Bereinigung an
print("\nWENDE BEREINIGUNG AN...")
df_aimeta['ai:dewey_clean'] = df_aimeta['ai:dewey'].apply(clean_dewey_comprehensive)

# Statistiken nach der Bereinigung
cleaned_count = sum(len(dewey_list) for dewey_list in df_aimeta['ai:dewey_clean'] if isinstance(dewey_list, list))
eliminated_total = original_count - cleaned_count

print(f"\nSTATISTIKEN NACH DER BEREINIGUNG:")
print(f"  Bereinigte Dewey-Einträge: {cleaned_count}")
print(f"  Eliminierte Einträge: {eliminated_total} ({eliminated_total/original_count*100:.2f}%)")

# Zeige Beispiele der Bereinigung
print(f"\n=== BEISPIELE DER LABEL-BEREINIGUNG ===")
examples_shown = 0
for dewey_list_orig, dewey_list_clean in zip(df_aimeta['ai:dewey'].head(20), df_aimeta['ai:dewey_clean'].head(20)):
    if isinstance(dewey_list_orig, list) and isinstance(dewey_list_clean, list):
        for orig_entry, clean_entry in zip(dewey_list_orig, dewey_list_clean):
            if (isinstance(orig_entry, dict) and isinstance(clean_entry, dict) and 
                'label' in orig_entry and 'label' in clean_entry and
                orig_entry['label'] != clean_entry['label']):
                
                print(f"  Original: '{orig_entry['label']}'")
                print(f"  Bereinigt: '{clean_entry['label']}'")
                print(f"  Notation: {clean_entry['notation']}")
                print()
                examples_shown += 1
                if examples_shown >= 5:
                    break
    if examples_shown >= 5:
        break

if examples_shown == 0:
    print("  Keine Label-Änderungen in den ersten 20 Datensätzen gefunden.")

print(f"✅ Bereinigte Daten stehen in 'ai:dewey_clean' zur Verfügung")
print(f"📝 Hinweis: Ursprüngliche Labels sind als 'original_label' in den Einträgen gespeichert")

=== UMFASSENDE DATENBEREINIGUNG ===

STATISTIKEN VOR DER BEREINIGUNG:
  Ursprüngliche Dewey-Einträge: 13486
  Labels mit Anführungszeichen: 0

WENDE BEREINIGUNG AN...

STATISTIKEN NACH DER BEREINIGUNG:
  Bereinigte Dewey-Einträge: 13481
  Eliminierte Einträge: 5 (0.04%)

=== BEISPIELE DER LABEL-BEREINIGUNG ===
  Keine Label-Änderungen in den ersten 20 Datensätzen gefunden.
✅ Bereinigte Daten stehen in 'ai:dewey_clean' zur Verfügung
📝 Hinweis: Ursprüngliche Labels sind als 'original_label' in den Einträgen gespeichert


In [37]:
# EXPORT INKONSISTENTER BEZEICHNUNGEN GRUPPENWEISE
print("=== EXPORT INKONSISTENTER BEZEICHNUNGEN NACH NOTATIONEN ===\n")

from collections import defaultdict, Counter
import pandas as pd

# Sammle alle bereinigten Dewey-Einträge
clean_entries = []
for dewey_list in df_aimeta['ai:dewey_clean']:
    if isinstance(dewey_list, list):
        for entry in dewey_list:
            if isinstance(entry, dict) and 'notation' in entry and 'label' in entry:
                clean_entries.append(entry)

# Gruppiere nach Notationen und identifiziere Inkonsistenzen
notation_groups = defaultdict(list)
for entry in clean_entries:
    notation_groups[entry['notation']].append(entry)

# Erstelle strukturierte Ausgabe für inkonsistente Notationen
inconsistent_groups = []

print(f"Analysiere {len(notation_groups)} verschiedene Notationen...")

for notation, entries in notation_groups.items():
    # Sammle alle Labels für diese Notation
    labels_in_notation = [entry['label'] for entry in entries]
    unique_labels = list(set(labels_in_notation))
    
    # Nur inkonsistente Notationen (mehrere verschiedene Labels)
    if len(unique_labels) > 1:
        # Zähle Häufigkeiten der Labels
        label_counts = Counter(labels_in_notation)
        total_count = len(entries)
        
        # Sammle Statistiken für jedes Label
        label_stats = []
        for label in unique_labels:
            count = label_counts[label]
            percentage = count / total_count * 100
            
            # Durchschnittlicher Score für dieses Label
            scores = [entry['score'] for entry in entries if entry['label'] == label]
            avg_score = sum(scores) / len(scores) if scores else 0
            
            label_stats.append({
                'label': label,
                'count': count,
                'percentage': percentage,
                'avg_score': avg_score
            })
        
        # Sortiere Labels nach Häufigkeit (häufigstes zuerst)
        label_stats.sort(key=lambda x: x['count'], reverse=True)
        
        inconsistent_groups.append({
            'notation': notation,
            'total_occurrences': total_count,
            'num_different_labels': len(unique_labels),
            'labels': label_stats
        })

# Sortiere Gruppen nach Gesamthäufigkeit (wichtigste zuerst)
inconsistent_groups.sort(key=lambda x: x['total_occurrences'], reverse=True)

print(f"Gefunden: {len(inconsistent_groups)} inkonsistente Notationen")

# Erstelle strukturierte Textdatei
output_file = "../dewey_evaluation/dewey_inconsistent_groups.txt"

with open(output_file, 'w', encoding='utf-8') as f:
    f.write("INKONSISTENTE DEWEY-KLASSIFIKATIONEN NACH NOTATIONEN\n")
    f.write("=" * 60 + "\n")
    f.write(f"Generiert am: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Anzahl inkonsistenter Notationen: {len(inconsistent_groups)}\n")
    f.write("=" * 60 + "\n\n")
    
    for i, group in enumerate(inconsistent_groups, 1):
        notation = group['notation']
        total = group['total_occurrences']
        num_labels = group['num_different_labels']
        
        f.write(f"{i:2d}. NOTATION: {notation}\n")
        f.write(f"    Gesamtvorkommen: {total}\n")
        f.write(f"    Verschiedene Labels: {num_labels}\n")
        f.write(f"    Labels (sortiert nach Häufigkeit):\n")
        f.write("-" * 50 + "\n")
        
        for j, label_info in enumerate(group['labels'], 1):
            label = label_info['label']
            count = label_info['count']
            percentage = label_info['percentage']
            avg_score = label_info['avg_score']
            
            f.write(f"    {j}. \"{label}\"\n")
            f.write(f"       Häufigkeit: {count:3d}x ({percentage:5.1f}%)\n")
            f.write(f"       Ø Score: {avg_score:.2f}\n")
            f.write("\n")
        
        f.write("\n" + "=" * 60 + "\n\n")

print(f"✅ Datei erstellt: {output_file}")

# Erstelle zusätzlich eine CSV-Datei für einfache Bearbeitung
csv_data = []
for group in inconsistent_groups:
    notation = group['notation']
    total = group['total_occurrences']
    
    for i, label_info in enumerate(group['labels']):
        csv_data.append({
            'notation': notation,
            'rank': i + 1,
            'label': label_info['label'],
            'count': label_info['count'],
            'percentage': label_info['percentage'],
            'avg_score': label_info['avg_score'],
            'total_for_notation': total,
            'num_alternatives': len(group['labels']),
            'is_dominant': i == 0  # Erstes (häufigstes) Label markieren
        })

df_inconsistent = pd.DataFrame(csv_data)
csv_file = "../dewey_evaluation/dewey_inconsistent_groups.csv"
df_inconsistent.to_csv(csv_file, index=False, encoding='utf-8')

print(f"✅ CSV-Datei erstellt: {csv_file}")

# Zeige Statistik-Übersicht
print(f"\n=== STATISTIK-ÜBERSICHT ===")
print(f"Top 10 problematischste Notationen:")
print("Rang | Notation | Vorkommen | Labels | Dominierendes Label (Anteil)")
print("-" * 75)

for i, group in enumerate(inconsistent_groups[:10], 1):
    notation = group['notation']
    total = group['total_occurrences']
    num_labels = group['num_different_labels']
    dominant = group['labels'][0]
    dominant_label = dominant['label'][:25]  # Kürze lange Labels
    dominant_pct = dominant['percentage']
    
    print(f"{i:3d}  | {notation:8s} | {total:9d} | {num_labels:6d} | {dominant_label:25s} ({dominant_pct:4.1f}%)")

print(f"\n💡 VERWENDUNG:")
print(f"   📄 Textdatei: '{output_file}' - Detaillierte Übersicht zum Lesen")
print(f"   📊 CSV-Datei: '{csv_file}' - Für Analyse in Excel/LibreOffice")
print(f"   🎯 Fokus: Notationen mit hohem 'total_for_notation' zuerst bearbeiten")
print(f"   ✅ Empfehlung: Meist das Label mit 'rank' = 1 (häufigstes) wählen")

=== EXPORT INKONSISTENTER BEZEICHNUNGEN NACH NOTATIONEN ===

Analysiere 202 verschiedene Notationen...
Gefunden: 38 inkonsistente Notationen
✅ Datei erstellt: ../dewey_evaluation/dewey_inconsistent_groups.txt
✅ CSV-Datei erstellt: ../dewey_evaluation/dewey_inconsistent_groups.csv

=== STATISTIK-ÜBERSICHT ===
Top 10 problematischste Notationen:
Rang | Notation | Vorkommen | Labels | Dominierendes Label (Anteil)
---------------------------------------------------------------------------
  1  | 004      |      1449 |      2 | Informatik                (99.9%)
  2  | 620      |       959 |      4 | Ingenieurwissenschaften u (99.6%)
  3  | 621      |       572 |      2 | Angewandte Physik         (99.8%)
  4  | 000      |       377 |      2 | Informatik, Information & (98.9%)
  5  | 410      |       161 |      2 | Linguistik                (99.4%)
  6  | 331      |       139 |      2 | Arbeitsökonomie           (99.3%)
  7  | 624      |       136 |      2 | Ingenieurbau              (99.3%)

In [38]:
# SIMULATION: VERWENDUNG DOMINANTER LABELS ZUR INKONSISTENZ-REDUKTION
print("=== SIMULATION: ANWENDUNG DOMINANTER LABELS ===\n")

# Erstelle eine Mapping-Tabelle der dominanten Labels pro Notation
dominant_mapping = {}
for group in inconsistent_groups:
    notation = group['notation']
    dominant_label = group['labels'][0]['label']  # Häufigstes Label
    dominant_mapping[notation] = dominant_label

print(f"Erstelle Mapping für {len(dominant_mapping)} inkonsistente Notationen...")

# Simuliere die Anwendung der dominanten Labels
def apply_dominant_labels(dewey_list):
    """
    Wendet dominante Labels auf inkonsistente Notationen an
    """
    if not isinstance(dewey_list, list):
        return dewey_list
    
    updated_list = []
    for entry in dewey_list:
        if isinstance(entry, dict) and 'notation' in entry and 'label' in entry:
            notation = entry['notation']
            
            # Wenn Notation inkonsistent ist, verwende dominantes Label
            if notation in dominant_mapping:
                updated_entry = entry.copy()
                updated_entry['original_label_before_dominance'] = entry['label']
                updated_entry['label'] = dominant_mapping[notation]
                updated_entry['was_updated_to_dominant'] = True
                updated_list.append(updated_entry)
            else:
                # Notation ist bereits konsistent
                entry_copy = entry.copy()
                entry_copy['was_updated_to_dominant'] = False
                updated_list.append(entry_copy)
        else:
            updated_list.append(entry)
    
    return updated_list

# Wende dominante Labels an
print("Wende dominante Labels an...")
df_aimeta['ai:dewey_with_dominant'] = df_aimeta['ai:dewey_clean'].apply(apply_dominant_labels)

# Analysiere die Ergebnisse nach der Anwendung
print("\n=== ANALYSE NACH ANWENDUNG DOMINANTER LABELS ===")

# Sammle alle Einträge nach der Aktualisierung
updated_entries = []
for dewey_list in df_aimeta['ai:dewey_with_dominant']:
    if isinstance(dewey_list, list):
        for entry in dewey_list:
            if isinstance(entry, dict) and 'notation' in entry and 'label' in entry:
                updated_entries.append(entry)

# Analysiere verbleibende Inkonsistenzen
updated_notation_groups = defaultdict(list)
for entry in updated_entries:
    updated_notation_groups[entry['notation']].append(entry)

# Zähle verbleibende Inkonsistenzen
remaining_inconsistent = []
for notation, entries in updated_notation_groups.items():
    labels_in_notation = [entry['label'] for entry in entries]
    unique_labels = list(set(labels_in_notation))
    
    if len(unique_labels) > 1:
        label_counts = Counter(labels_in_notation)
        remaining_inconsistent.append({
            'notation': notation,
            'labels': unique_labels,
            'counts': dict(label_counts),
            'total': len(entries)
        })

# Statistiken vor und nach der Anwendung
print(f"ERGEBNISSE:")
print(f"  Inkonsistenzen VOR dominanter Label-Anwendung: {len(inconsistent_groups)}")
print(f"  Inkonsistenzen NACH dominanter Label-Anwendung: {len(remaining_inconsistent)}")
print(f"  Reduzierte Inkonsistenzen: {len(inconsistent_groups) - len(remaining_inconsistent)}")
print(f"  Reduktion: {(len(inconsistent_groups) - len(remaining_inconsistent)) / len(inconsistent_groups) * 100:.1f}%")

# Zähle aktualisierte Einträge
updated_count = sum(1 for entry in updated_entries if entry.get('was_updated_to_dominant', False))
total_entries = len(updated_entries)

print(f"\nDETAILS:")
print(f"  Gesamte Dewey-Einträge: {total_entries}")
print(f"  Auf dominantes Label aktualisiert: {updated_count}")
print(f"  Aktualisierungsrate: {updated_count / total_entries * 100:.2f}%")

# Zeige verbleibende Inkonsistenzen (falls vorhanden)
if remaining_inconsistent:
    print(f"\n⚠️  VERBLEIBENDE INKONSISTENZEN:")
    print("Diese Notationen haben auch nach der Anwendung dominanter Labels noch Inkonsistenzen:")
    print("(Dies kann durch zeitgleiche Updates oder andere Faktoren entstehen)")
    print("-" * 70)
    
    for i, inconsistency in enumerate(remaining_inconsistent[:10], 1):  # Zeige nur die ersten 10
        notation = inconsistency['notation']
        labels = inconsistency['labels']
        counts = inconsistency['counts']
        total = inconsistency['total']
        
        print(f"{i:2d}. Notation {notation} ({total} Einträge):")
        for label in labels:
            count = counts[label]
            percentage = count / total * 100
            print(f"    - \"{label[:40]}...\" ({count}x, {percentage:.1f}%)")
        print()
else:
    print(f"\n✅ PERFEKTE KONSISTENZ ERREICHT!")
    print("Alle Inkonsistenzen wurden durch die Anwendung dominanter Labels beseitigt.")

# Erstelle Übersicht der Notation-Konsistenz
all_notations = set(updated_notation_groups.keys())
consistent_notations = all_notations - {inc['notation'] for inc in remaining_inconsistent}

print(f"\n=== KONSISTENZ-ÜBERSICHT ===")
print(f"Gesamte verschiedene Notationen: {len(all_notations)}")
print(f"Konsistente Notationen: {len(consistent_notations)} ({len(consistent_notations)/len(all_notations)*100:.1f}%)")
print(f"Inkonsistente Notationen: {len(remaining_inconsistent)} ({len(remaining_inconsistent)/len(all_notations)*100:.1f}%)")

# Empfehlung
print(f"\n💡 EMPFEHLUNG:")
if len(remaining_inconsistent) == 0:
    print("   ✅ Die Anwendung dominanter Labels löst alle Inkonsistenzen!")
    print("   🎯 Nächster Schritt: Implementierung der automatischen Label-Vereinheitlichung")
elif len(remaining_inconsistent) < len(inconsistent_groups) * 0.1:  # Weniger als 10% verbleibend
    print(f"   ✅ Sehr gute Reduktion auf nur {len(remaining_inconsistent)} verbleibende Inkonsistenzen")
    print("   🎯 Manuelle Überprüfung der wenigen verbleibenden Fälle empfohlen")
else:
    print(f"   ⚠️  Noch {len(remaining_inconsistent)} Inkonsistenzen verbleibend")
    print("   🎯 Weitere Analyse der verbleibenden Fälle notwendig")

print(f"\n📊 Die simulierten Daten stehen in 'ai:dewey_with_dominant' zur Verfügung")

=== SIMULATION: ANWENDUNG DOMINANTER LABELS ===

Erstelle Mapping für 38 inkonsistente Notationen...
Wende dominante Labels an...

=== ANALYSE NACH ANWENDUNG DOMINANTER LABELS ===
ERGEBNISSE:
  Inkonsistenzen VOR dominanter Label-Anwendung: 38
  Inkonsistenzen NACH dominanter Label-Anwendung: 0
  Reduzierte Inkonsistenzen: 38
  Reduktion: 100.0%

DETAILS:
  Gesamte Dewey-Einträge: 13481
  Auf dominantes Label aktualisiert: 4331
  Aktualisierungsrate: 32.13%

✅ PERFEKTE KONSISTENZ ERREICHT!
Alle Inkonsistenzen wurden durch die Anwendung dominanter Labels beseitigt.

=== KONSISTENZ-ÜBERSICHT ===
Gesamte verschiedene Notationen: 202
Konsistente Notationen: 202 (100.0%)
Inkonsistente Notationen: 0 (0.0%)

💡 EMPFEHLUNG:
   ✅ Die Anwendung dominanter Labels löst alle Inkonsistenzen!
   🎯 Nächster Schritt: Implementierung der automatischen Label-Vereinheitlichung

📊 Die simulierten Daten stehen in 'ai:dewey_with_dominant' zur Verfügung


In [ ]:
# SUNBURST-DIAGRAMM DER HIERARCHISCHEN DEWEY-KLASSIFIKATIONEN
print("=== ERSTELLE SUNBURST-DIAGRAMM ===\n")

import plotly.graph_objects as go
import plotly.express as px
from collections import defaultdict, Counter

# Sammle alle konsistenten Einträge aus der Simulation
consistent_entries = []
for dewey_list in df_aimeta['ai:dewey_with_dominant']:
    if isinstance(dewey_list, list):
        for entry in dewey_list:
            if isinstance(entry, dict) and 'notation' in entry and 'label' in entry:
                consistent_entries.append(entry)

print(f"Verwende {len(consistent_entries)} konsistente Dewey-Einträge für das Diagramm")

# Funktion zur Hierarchie-Extraktion aus Dewey-Notationen
def extract_dewey_hierarchy(notation):
    """
    Extrahiert hierarchische Ebenen aus Dewey-Notationen
    Beispiel: "621.38" -> ["600", "620", "621", "621.3", "621.38"]
    """
    if not notation or not str(notation).replace('.', '').isdigit():
        return []
    
    notation_str = str(notation).strip()
    
    # Trenne Ganzzahl und Dezimalteil
    if '.' in notation_str:
        main_part, decimal_part = notation_str.split('.', 1)
    else:
        main_part = notation_str
        decimal_part = ""
    
    # Erstelle Hierarchie-Ebenen
    hierarchy = []
    
    # Ebene 1: Hauptklassen (100, 200, 300, ...)
    if len(main_part) >= 1:
        first_digit = main_part[0]
        if first_digit != '0':
            hierarchy.append(first_digit + "00")
    
    # Ebene 2: Zweite Ebene (110, 120, 130, ...)
    if len(main_part) >= 2:
        second_level = main_part[:2] + "0"
        if second_level != hierarchy[-1] if hierarchy else True:
            hierarchy.append(second_level)
    
    # Ebene 3: Dritte Ebene (111, 112, 113, ...)
    if len(main_part) >= 3:
        third_level = main_part
        if third_level != hierarchy[-1] if hierarchy else True:
            hierarchy.append(third_level)
    
    # Ebene 4+: Dezimalstellen (621.3, 621.38, ...)
    if decimal_part:
        for i in range(1, len(decimal_part) + 1):
            decimal_level = main_part + "." + decimal_part[:i]
            hierarchy.append(decimal_level)
    
    return hierarchy

# Sammle Hierarchie-Daten
hierarchy_counts = defaultdict(int)
label_mapping = {}

for entry in consistent_entries:
    notation = entry['notation']
    label = entry['label']
    
    # Extrahiere Hierarchie
    hierarchy = extract_dewey_hierarchy(notation)
    
    # Zähle jede Hierarchie-Ebene
    for level_notation in hierarchy:
        hierarchy_counts[level_notation] += 1
        
        # Speichere Label für diese Notation (nehme das dominante)
        if level_notation not in label_mapping:
            label_mapping[level_notation] = label
        elif level_notation == notation:  # Für die finale Ebene, nehme das aktuelle Label
            label_mapping[level_notation] = label

print(f"Extrahierte {len(hierarchy_counts)} verschiedene Hierarchie-Ebenen")

# Erstelle Sunburst-Datenstruktur
sunburst_data = []

# Sortiere Notationen nach Länge (von allgemein zu spezifisch)
sorted_notations = sorted(hierarchy_counts.keys(), key=lambda x: (len(x.replace('.', '')), x))

for notation in sorted_notations:
    count = hierarchy_counts[notation]
    label = label_mapping.get(notation, f"Dewey {notation}")
    
    # Bestimme Parent (übergeordnete Ebene)
    parent = ""
    notation_str = str(notation)
    
    if '.' in notation_str:
        # Für Dezimalstellen: Parent ist ohne letzte Dezimalstelle
        main, decimal = notation_str.split('.', 1)
        if len(decimal) > 1:
            parent = main + '.' + decimal[:-1]
        else:
            parent = main
    else:
        # Für Ganzzahlen: Parent ist kürzere Notation
        if len(notation_str) == 3 and notation_str.endswith('0'):
            if notation_str[1] == '0':
                # 100, 200, 300 -> keine Parent (Hauptklassen)
                parent = ""
            else:
                # 110, 120, 130 -> Parent ist 100, 200, etc.
                parent = notation_str[0] + "00"
        elif len(notation_str) == 3:
            # 111, 112, 113 -> Parent ist 110, 120, etc.
            parent = notation_str[:2] + "0"
    
    # Funktion für automatische Zeilenumbrüche bei langen Labels
    def format_label_with_breaks(text, max_length=25):
        """Fügt Zeilenumbrüche in langen Labels ein"""
        words = text.split()
        lines = []
        current_line = []
        current_length = 0
        
        for word in words:
            if current_length + len(word) + 1 <= max_length:
                current_line.append(word)
                current_length += len(word) + 1
            else:
                if current_line:
                    lines.append(' '.join(current_line))
                current_line = [word]
                current_length = len(word)
        
        if current_line:
            lines.append(' '.join(current_line))
        
        return '<br>'.join(lines)
    
    # Erstelle formatiertes Label mit Dewey-Nummer und umbrochenen Labels
    formatted_label = format_label_with_breaks(label)
    display_label = f"<b>{notation}</b><br>{formatted_label}<br><i>({count:,} Einträge)</i>"
    
    sunburst_data.append({
        'ids': notation,
        'labels': display_label,
        'parents': parent,
        'values': count
    })

# Sortiere Daten für bessere Darstellung
sunburst_data.sort(key=lambda x: (len(str(x['parents'])), str(x['parents']), len(str(x['ids'])), str(x['ids'])))

print(f"Erstelle Sunburst mit {len(sunburst_data)} Segmenten")

# Erstelle Sunburst-Diagramm
fig = go.Figure(go.Sunburst(
    ids=[item['ids'] for item in sunburst_data],
    labels=[item['labels'] for item in sunburst_data],
    parents=[item['parents'] for item in sunburst_data],
    values=[item['values'] for item in sunburst_data],
    branchvalues="total",
    hovertemplate='<b>Dewey %{id}</b><br>%{label}<br><b>Anzahl: %{value:,}</b><extra></extra>',
    maxdepth=4,  # Begrenze Tiefe für bessere Lesbarkeit
))

fig.update_layout(
    title={
        'text': "Hierarchische Dewey-Klassifikationen (Konsistente Daten)<br><sub>Innerer Kreis: Hauptklassen (100, 200, ...) → Mittlerer Kreis: Unterklassen (110, 120, ...) → Äußerer Kreis: Spezifische Klassen</sub>",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    font_size=9,
    width=1000,
    height=800,
    margin=dict(t=100, b=50, l=50, r=50)
)

# Zeige Diagramm
fig.show()

# HOCHAUFGELÖSTER EXPORT ALS BILD
print("\n=== SPEICHERE HOCHAUFGELÖSTES BILD ===")

# Export als PNG (hohe Qualität für Präsentationen)
try:
    output_file_png = "dewey_sunburst_highres.png"
    fig.write_image(
        output_file_png,
        format="png",
        width=3000,      # Breite in Pixeln
        height=2400,     # Höhe in Pixeln
        scale=2          # Skalierungsfaktor (effektiv: 6000x4800 Pixel)
    )
    print(f"✅ PNG gespeichert: {output_file_png}")
    print(f"   Auflösung: 6000x4800 Pixel (effektiv durch scale=2)")
except Exception as e:
    print(f"❌ PNG-Export fehlgeschlagen: {e}")
    print("   Installiere kaleido: pipenv install kaleido")

# Export als PDF (Druckqualität)
try:
    output_file_pdf = "dewey_sunburst_print.pdf"
    fig.write_image(
        output_file_pdf,
        format="pdf",
        width=3000,
        height=2400
    )
    print(f"✅ PDF gespeichert: {output_file_pdf}")
    print(f"   Format: Druckqualität")
except Exception as e:
    print(f"❌ PDF-Export fehlgeschlagen: {e}")
    print("   Installiere kaleido: pipenv install kaleido")

# Statistiken zur Hierarchie
print(f"\n=== HIERARCHIE-STATISTIKEN ===")

# Zähle Ebenen
level_counts = defaultdict(int)
for notation in hierarchy_counts.keys():
    if '.' in str(notation):
        decimal_places = len(str(notation).split('.')[1])
        level = 3 + decimal_places  # 3 für Ganzzahl-Teil + Dezimalstellen
    else:
        level = len(str(notation).replace('0', ''))  # Anzahl nicht-null Stellen
        level = min(level, 3)  # Maximal 3 für Ganzzahl-Teil
    
    level_counts[level] += 1

print("Verteilung nach Hierarchie-Ebenen:")
for level in sorted(level_counts.keys()):
    count = level_counts[level]
    if level == 1:
        print(f"  Ebene {level} (Hauptklassen 100, 200, ...): {count} Klassen")
    elif level == 2:
        print(f"  Ebene {level} (Unterklassen 110, 120, ...): {count} Klassen")
    elif level == 3:
        print(f"  Ebene {level} (Spezifische Klassen 111, 112, ...): {count} Klassen")
    else:
        print(f"  Ebene {level} (Dezimalklassen): {count} Klassen")

# Top 10 meist verwendete Hauptklassen
main_classes = {}
for notation, count in hierarchy_counts.items():
    if len(str(notation)) == 3 and str(notation).endswith('00'):
        main_classes[notation] = count

print(f"\nTop 10 Hauptklassen (nach Häufigkeit):")
for i, (notation, count) in enumerate(sorted(main_classes.items(), key=lambda x: x[1], reverse=True)[:10], 1):
    label = label_mapping.get(notation, f"Dewey {notation}")
    percentage = count / sum(main_classes.values()) * 100
    print(f"  {i:2d}. {notation} - {label[:40]}: {count:,} ({percentage:.1f}%)")

print(f"\n✅ Sunburst-Diagramm erstellt mit konsistenten Dewey-Daten")